In [1]:
import extruct
import requests
from pprint import pprint
from fake_useragent import UserAgent
from w3lib.html import get_base_url
import mongoengine

# Import user agent
ua = UserAgent()

In [2]:
from mongoengine import connect
connect('track')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

# Shopee

In [3]:
headers = {
    'User-Agent': 'Mozilla/5',
     'Referer': 'https://shopee.sg/search?keyword=instant+noodles'
}

url = 'https://shopee.sg/api/v2/search_items/?by=relevancy&keyword=instant+noodles&limit=50&newest=0&order=desc&page_type=search'  
r = requests.get(url, headers = headers).json()
r['items']

[{'itemid': 4018081570,
  'welcome_package_info': None,
  'liked': False,
  'recommendation_info': None,
  'bundle_deal_info': None,
  'price_max_before_discount': -1,
  'image': 'face8662413ed0eddc243e4c50e1c19e',
  'is_cc_installment_payment_eligible': False,
  'shopid': 68603365,
  'can_use_wholesale': False,
  'group_buy_info': None,
  'reference_item_id': '',
  'currency': 'SGD',
  'raw_discount': 0,
  'show_free_shipping': False,
  'video_info_list': [],
  'ads_keyword': 'noodles',
  'collection_id': None,
  'images': ['face8662413ed0eddc243e4c50e1c19e',
   'f1c639b32f39e924ccf998757899642c',
   '5ee4aba5fb9e682d69917dc5a529410f',
   'd1b4c5284b779fca0c45764ce4fa40fd',
   '6a623f9407721ddf23cf68d05736ef5a',
   '1fdfd7ff7ac46518a307f5bca860a74a',
   'a9f111effa0958d3148d640e2e57c34b',
   '89e68f2249e13e26c1e4dbda60740aab',
   'efe3880c62e91eff6b6a1cf09ec6f4b5'],
  'match_type': 1,
  'price_before_discount': 0,
  'is_category_failed': False,
  'show_discount': 0,
  'cmt_count': 1,


In [7]:
SHOPEE_PHOTO_URL = 'https://cf.shopee.sg/file/'

class Variation(mongoengine.EmbeddedDocument):
    variation_name = mongoengine.StringField()
    variation_options = mongoengine.ListField()

class Wholesale(mongoengine.EmbeddedDocument):  
    wholesale_min = mongoengine.IntField()
    wholesale_max = mongoengine.IntField()
    wholesale_price = mongoengine.DecimalField()

class History(mongoengine.EmbeddedDocument):  
    history_min = mongoengine.DecimalField()
    history_max = mongoengine.DecimalField()
    history_timestamp = mongoengine.DateTimeField()    
    
class Platform(mongoengine.EmbeddedDocument):
    platform = mongoengine.StringField()
    platform_item_id = mongoengine.StringField()
    platform_shop_id = mongoengine.StringField()
    platform_shop_location = mongoengine.StringField()
    platform_item_brand = mongoengine.StringField()
    platform_item_name = mongoengine.StringField()
    platform_item_stock = mongoengine.IntField()
    platform_item_sold = mongoengine.IntField()
    platform_item_views = mongoengine.IntField()
    platform_item_likes = mongoengine.IntField()
    platform_item_ratings = mongoengine.ListField()
    platform_item_ratings_count = mongoengine.IntField()    
    platform_item_ratings_average = mongoengine.DecimalField() 
    platform_item_free_shipping = mongoengine.BooleanField() 
    platform_item_images = mongoengine.ListField()
    platform_item_variations = mongoengine.EmbeddedDocumentListField(Variation)
    platform_item_currency = mongoengine.StringField()
    platform_item_wholesale = mongoengine.EmbeddedDocumentListField(Wholesale)
    platform_item_bundle = mongoengine.ListField()
    platform_current_price_min = mongoengine.DecimalField()
    platform_current_price_max = mongoengine.DecimalField()
    platform_price_history = mongoengine.EmbeddedDocumentListField(History) 
    
class Item(mongoengine.Document):
    name = mongoengine.StringField()
    photos = mongoengine.ListField()
    platforms = mongoengine.EmbeddedDocumentListField(Platform)
    
item = Item(name="Acure", photos=['fdf2edf39918a945ea308c567c73c9fd',
   'd8774e2cc906aad7f5b5f838f51f67eb'])

shopee_item = Platform(platform_item_id="asd212", platform_item_sold=20)
item.platforms = [shopee_item]
item.save()

<Item: Item object>

In [33]:

for item in r['items']:
    data = {}
    # general product attributes
    data['name'] = item['name']
    data['photos'] = [SHOPEE_PHOTO_URL + s for s in item['images']]
    # platform product attributes
    data['platforms'] = ['shopee']
    data['platforms'][0] = "asd"
    data['platforms']['shopee']['itemid'] = item['itemid']
    result = db.test.insert_one(data)




# search for product using fuzzy matching
# if no match, add product
    # name, photo links
# add shop-product attributes
    # itemid
    # shopid
    # shop_location
    # brand
    # name
    # historical_sold
    # view_count
    # liked_count
    # sold
    # item_rating
        # ['item_rating']['rating_star']
        # ['item_rating']['rating_count']
            #'rating_count': [154, 1, 3, 1, 9, 140]
    # stock
    # show_free_shipping
    # images
        # 'images'
    # variations
        # ['tier_variations'][:]['name']
        # ['tier_variations'][:]['options']
    # currency
        # 'currency'
    # wholesale price
        # 'wholesale_tier_list': [{'min_count': 3, 'price': 550000, 'max_count': 5},
        #{'min_count': 6, 'price': 400000, 'max_count': None}],
    # bundle_deal_info
        # if ['bundle_deal_info'] is not None
        # ['bundle_deal_info']['bundle_deal_label']
    # current price
        # price_min, price_max
        # if scraped price = current price, no edits, else update
    # price history
        # if scraped price = current price, no edits, else add price and current timestamp to array
    

TypeError: list indices must be integers or slices, not str

In [9]:
from IPython.display import Image
from IPython.core.display import HTML 
#  photo URL
shopee_photo_link = 'https://cf.shopee.sg/file/'
Image(url= shopee_photo_link+"44d2dbc3e358671c832b48f112d8c23f", width=50, height=50)

# Qoo10

In [11]:
headers = {
    'User-Agent': 'Mozilla/5',
#      'Referer': 'https://shopee.sg/search?keyword=Acure+Organics'
}

url = 'https://www.qoo10.sg/s/ACURE-ORGANICS?keyword=acure+organics&keyword_auto_change='  
r = requests.get(url, headers = headers)

In [12]:
pprint(r.text)

('\r\n'
 '<!DOCTYPE html>\r\n'
 '<html lang="en">\r\n'
 '<head id="ctl00_ctl00_Head2"><title>\r\n'
 '\tQoo10 - acure organics Search Results : (Q·Ranking)： Items now on sale at '
 'qoo10.sg\r\n'
 '</title><meta charset="utf-8" />\r\n'
 '\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n'
 '\t<meta name="application-name" content="Qoo10.sg - SG No.1 Shopping '
 'Destination." />\r\n'
 '    <meta name="format-detection" content="no" /><meta '
 'http-equiv="x-dns-prefetch-control" content="on" /><link rel="dns-prefetch" '
 'href="//dp.image-gmkt.com"/><link rel="dns-prefetch" '
 'href="//gd.image-gmkt.com"/><link rel="dns-prefetch" '
 'href="//stsg-b.image-gmkt.com"/><link rel="dns-prefetch" '
 'href="//gdetail.image-gmkt.com"/><link rel="dns-prefetch" '
 'href="//www.qoo10.sg"/><link id="ctl00_ctl00_shortcut_icon" rel="shortcut '
 'icon" '
 'href="//stsg-b.image-gmkt.com/css/sg/qoo10/front/cm/common/image/qoo10.v_20120529.ico" '
 '/><link id="ctl00_ctl00_webapp_manifest" rel="

 'onclick="if(window.AutoSearchLayer && AutoSearchLayer.close) '
 'AutoSearchLayer.close();">Delete layer</button>\r\n'
 '\t\t\t\t\t</div>\r\n'
 '\t\t\t\t</div>\r\n'
 '\t\t\t\t\r\n'
 '\t\t\t</div>\r\n'
 '\r\n'
 '\t\t\t\r\n'
 '\t\t\t<ul class="head_key">\r\n'
 '\t\t\t\t<li class="my">\r\n'
 '\t\t\t\t\t<a href="https://www.qoo10.sg/gmkt.inc/My/Default.aspx" '
 'onclick="if (window.clickMyPageMove) clickMyPageMove(this, \'my_layer\'); '
 'return false;">My Qoo10</a>\r\n'
 '\t\t\t\t\t\r\n'
 '\t\t\t\t\t\r\n'
 '\t\t\t\t\t<div class="head_key_dt" style="display:none;">\t\t<ul>\t\t<li><a '
 'href="https://www.qoo10.sg/gmkt.inc/My/OrderContractList.aspx">Recent '
 'Orders</a></li>\t\t<li><a '
 'href="https://www.qoo10.sg/gmkt.inc/My/OrderContractList.aspx?tab_type=CR">Cancel/Refund</a></li>\t\t'
 '<li><a href="https://www.qoo10.sg/gmkt.inc/My/WishList.aspx">Wish '
 'List</a></li>\t\t<li><a '
 'href="https://www.qoo10.sg/gmkt.inc/My/SellerShop.aspx">Following '
 'Shop</a></li>\t\t<li><a href="#"

 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000047&amp;gdmc_cd=200000068">Golf '
 '/ Ski</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000047&amp;gdmc_cd=220000034">MMA '
 '/ Martial Arts</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000047&amp;gdmc_cd=200000343">Soccer '
 '/ Other Sports</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000047&amp;gdmc_cd=200000365">Sport '
 'Supplements</a></li></ul><div class="lnk"><a '
 'href="https://www.qoo10.sg/sp/193855">#Father\'s Day Sale</a><a '
 'href="https://www.qoo10.sg/sp/154829">#Fitness Factory</a><a '
 'href="https://www.qoo10.sg/sp/191559">#Gentlemen\'s '
 'Lair</a></div></div></div><div class="tmp_nv_ctg" '
 'data-type="orgin_category_layer" data-group_code="4" '
 'style="display:none;"><div class="grp"><h4><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/Default.aspx?gdlc_cd=100000014">Mobile '
 'Devices</a></h4><ul><li><a '
 'h

 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000016&amp;gdmc_cd=200000110">Boy\'s '
 'Clothing</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000016&amp;gdmc_cd=200000111">Etc '
 'Clothing</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000016&amp;gdmc_cd=200000112">Bags '
 '/ Shoes</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000016&amp;gdmc_cd=200000113">Hats '
 '/ Socks</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000016&amp;gdmc_cd=200000114">Underwears '
 '/ PJs</a></li><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000016&amp;gdmc_cd=200000115">Accessories '
 '/ More</a></li></ul></div><div class="grp"><h4><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000050">Toys</a></h4><ul><li><a '
 'href="https://www.qoo10.sg/gmkt.inc/Category/?gdlc_cd=100000050&amp;gdmc_cd=200000354">Robots/Vehicles</a></li><li><a '
 'href

 'onclick="if(window.setSearchValues) setSearchValues(\'shipto\', this)" '
 'val="SG"><dfn class="flag_sg"></dfn><span>Singapore</span></a></li><li><a '
 'onclick="if(window.setSearchValues) setSearchValues(\'shipto\', this)" '
 'val="AE"><dfn class="flag_ae"></dfn><span>Arab '
 'Emirates</span></a></li><li><a onclick="if(window.setSearchValues) '
 'setSearchValues(\'shipto\', this)" val="AU"><dfn '
 'class="flag_au"></dfn><span>Australia</span></a></li><li><a '
 'onclick="if(window.setSearchValues) setSearchValues(\'shipto\', this)" '
 'val="AT"><dfn class="flag_at"></dfn><span>Austria</span></a></li><li><a '
 'onclick="if(window.setSearchValues) setSearchValues(\'shipto\', this)" '
 'val="BE"><dfn class="flag_be"></dfn><span>BELGIUM</span></a></li><li><a '
 'onclick="if(window.setSearchValues) setSearchValues(\'shipto\', this)" '
 'val="BN"><dfn class="flag_bn"></dfn><span>Brunei</span></a></li><li><a '
 'onclick="if(window.setSearchValues) setSearchValues(\'shipto\', this)" '
 'val=

...

 'href="https://www.qoo10.sg/item/QPRIME-ACURE-ORGANICS-MARULA-OIL-TREATMENT-ALL-SKIN-TYPES-1/618278954" '
 'title="[Qprime]Acure Organics Marula Oil Treatment All Skin Types 1 fl oz '
 '(30 ml)" target="_blank" data-type="goods_url">[Qprime]Acure Organics Marula '
 'Oil Treatment All Skin Types 1 fl oz (30 ml)</a></div><div '
 'class="opt_dtl"><a href="#none" class="wsh" data-type="btn_list_view_wish" '
 'title="">wished</a><a '
 'href="https://www.qoo10.sg/item/QPRIME-ACURE-ORGANICS-MARULA-OIL-TREATMENT-ALL-SKIN-TYPES-1/618278954" '
 'class="lnk_vw" onclick="if(window.AutoQuickView && '
 "AutoQuickView.onmouseover) AutoQuickView.onmouseover('618278954', 0); return "
 'false;">Quick View</a><a '
 'href="https://www.qoo10.sg/shop/Dyoninternational?cit=618278954" '
 'target="_blank" title="D.Y.O.N International" class="lnk_sh"><span '
 'class="grd_pw" title="Power Seller">Power seller</span>D.Y.O.N '
 'Internationa...</a></div></div></td><td class="td_prc"><div '
 'class="prc"><strong>S

 'class="sbj"><a '
 'href="https://www.qoo10.sg/item/ACURE-ORGANICS%ef%bc%8c-CLARIFYING-SHAMPOO%ef%bc%8c-LEMONGRASS-ARGAN-STEM-CELL%ef%bc%8c/552161102" '
 'title="Acure Organics， Clarifying Shampoo， Lemongrass + Argan Stem Cell， 8 '
 'fl oz (235 ml)" target="_blank" data-type="goods_url">Acure Organics， '
 'Clarifying Shampoo， Lemongrass + Argan Stem Cell， 8 fl oz (235 '
 'ml)</a></div><div class="opt_dtl"><a href="#none" class="wsh" '
 'data-type="btn_list_view_wish" title="">wished</a><a '
 'href="https://www.qoo10.sg/item/ACURE-ORGANICS%ef%bc%8c-CLARIFYING-SHAMPOO%ef%bc%8c-LEMONGRASS-ARGAN-STEM-CELL%ef%bc%8c/552161102" '
 'class="lnk_vw" onclick="if(window.AutoQuickView && '
 "AutoQuickView.onmouseover) AutoQuickView.onmouseover('552161102', 0); return "
 'false;">Quick View</a><a '
 'href="https://www.qoo10.sg/shop/Best_price?cit=552161102" target="_blank" '
 'title="House of [Cetaphil]" class="lnk_sh"><span class="grd_pw" title="Power '
 'Seller">Power seller</span>House of [Cetap

 'class="td_ship"><div class="ship_area"><div class="ship"><dfn>Shipping '
 'rate:</dfn> S$3.99</div><div class="ship_con fs_11">Free on condition '
 'S$95.00</div><div class="shp_ntn" title="Singapore"><dfn '
 'class="flag_sg"></dfn>SG</div></div></td><td><span class="rate_v" '
 'title="Rating: (0/5)"><span class="" style=height:0%;>Rating: '
 '(0/5)</span></span></td></tr>\r\n'
 '\t\t\t\t\t\t\r\n'
 '\t\t\t\t\t\t\t<tr id="g_538628323" goodscode=538628323 '
 'list_type="search_new_list_type"><td class="td_thmb"><div class="inner"><a '
 'href="https://www.qoo10.sg/item/ACURE-ORGANICS-FACIAL-CLEANSING-GEL-SUPERFRUIT-CHLORELLA-GROWTH/538628323" '
 'title="Acure Organics Facial Cleansing Gel SuperFruit + Chlorella Growth '
 'Factor 4 fl oz (118 ml)" target="_blank" !#MOUSEOVER_IMG_CUT#! '
 'data-type="goods_url"><img '
 'src="https://dp.image-gmkt.com/GMKT.IMG/loading_2017/qoo10_loading.s_100-w-fs_s.v_20170420.png" '
 'gd_src="https://gd.image-gmkt.com/li/569/013/789013569.g_100-w-st_g.jpg

 '=1348885481 img_cnt =3 globalcheck =N data-type="goods_url"><img '
 'src="https://dp.image-gmkt.com/GMKT.IMG/loading_2017/qoo10_loading.s_100-w-fs_s.v_20170420.png" '
 'gd_src="https://gd.image-gmkt.com/li/481/885/1348885481.g_100-w-st_g.jpg" '
 'load="N" resizing="false" width="100" height="110" alt="▶$1 Shop Coupon◀  '
 'Acure Organics Seriously Soothing Blue Tansy Night Oil (Pack of 2) With Blue '
 'Tansy"></a></div></td><td class="td_item"><div class="inner"><div '
 'class="sbj"><a '
 'href="https://www.qoo10.sg/item/1-SHOP-COUPON-ACURE-ORGANICS-SERIOUSLY-SOOTHING-BLUE-TANSY-NIGHT/667947771" '
 'title="▶$1 Shop Coupon◀  Acure Organics Seriously Soothing Blue Tansy Night '
 'Oil (Pack of 2) With Blue Tansy" target="_blank" data-type="goods_url">▶$1 '
 'Shop Coupon◀  Acure Organics Seriously Soothing Blue Tansy Night Oil (Pack '
 'of 2) With Blue Tansy</a></div><div class="opt_dtl"><a href="#none" '
 'class="wsh" data-type="btn_list_view_wish" title="">wished</a><a '
 'href="https:

 'src="//stsg-b.image-gmkt.com/js3/sg/gmkt.inc/control/list/scripts/categorysearchresult.min.v_20180614145335.js"></script>\n'
 '<script '
 'src="//stsg-b.image-gmkt.com/js3/sg/gmkt.inc/control/ui/scripts/uisearchfilter.min.v_20190605183801.js"></script>\n'
 '<script '
 'src="//stsg-b.image-gmkt.com/js3/sg/gmkt.inc/control/ui/scripts/uisearchlist.min.v_20190827125646.js"></script>\r\n'
 '\r\n'
 '<script>/* <![CDATA[ */\n'
 'function __initDefaultValue(){};\n'
 'if(!window.__ClientResource)window.__ClientResource=new '
 'Array();__ClientResource["click to fold"]="fold";__ClientResource["click to '
 'open"]="click to '
 'open";__ClientResource["close"]="Close";__ClientResource["ddayhour"]="9";__ClientResource["initialize"]="Initialize";__ClientResource["seemorebrand"]="See '
 'More Brand";__ClientResource["__master/defaultmasterpage.master__sign '
 'out"]="Sign out";__ClientResource["__master/defaultmasterpage.master__sign '
 'in"]="Sign '
 'in";__ClientResource["__master/defaultmasterpa

In [15]:
headers = {
    'User-Agent': 'Mozilla/5',
#      'Referer': 'https://shopee.sg/search?keyword=Acure+Organics'
}

url = 'https://sg.iherb.com/search?sug=acure+organics&kw=acure+organics&rank=0&noi=96'  
r = requests.get(url, headers = headers)

pprint(r.text)

('\n'
 '\n'
 '<!DOCTYPE html>\n'
 '\n'
 '\n'
 '\n'
 '\n'
 '<!--[if IE 9 ]> <html class="ie9 en search index " dir="ltr" lang="en"> '
 '<![endif]-->\n'
 '<!--[if IE 10 ]> <html class="ie10 en search index " dir="ltr" lang="en"> '
 '<![endif]-->\n'
 '<!--[if (gt IE 9)|!(IE)]><!-->\n'
 '\n'
 '\n'
 '<html class="ih-public ih-catalog cc-sg en search index search  core-app" '
 'dir="ltr" lang="en">\n'
 '<!--<![endif]-->\n'
 '<head>\n'
 '    <link rel="icon" type="image/ico" '
 'href="https://s3.images-iherb.com/static/i/favicon-iherb/favicon.ico" />\n'
 '\n'
 '    <!-- production/catalog/taiwan 644 -->\n'
 '\n'
 '    \n'
 '\n'
 '    <link rel="dns-prefetch" href="https://s3.images-iherb.com/">\n'
 '\n'
 '    \n'
 '    \n'
 '\n'
 '    <script>\n'
 '        var iHerb_Protocol = location.protocol;\n'
 '        var iHerb_BaseUrl = location.host;\n'
 '\n'
 '        var isLoveLetter = false;\n'
 '        window.PRODUCT_LIST = [];\n'
 '         var translations = {\n'
 '             btnAddToCart: "

 '0.454 0.19 0.589 0.183 0.133 0.431 0.133 0.614 0l7.724-5.641 7.709 '
 '5.641c0.092 0.067 0.199 0.101 0.307 0.101 0.107 0 0.215-0.033 0.307-0.101 '
 '0.184-0.135 0.26-0.371 0.19-0.589l-2.958-9.134 7.692-5.623c0.183-0.133 '
 '0.26-0.371 0.19-0.588z"></path>\n'
 '            <path class="rated" d="M27.436 '
 '10.888c-0.070-0.216-0.271-0.363-0.497-0.363h-9.501l-2.941-9.084c-0.070-0.216-0.271-0.363-0.497-0.364s-0.427 '
 '0.147-0.497 0.362l-2.957 9.085h-9.485c-0.226 0-0.427 0.148-0.497 0.363s0.007 '
 '0.454 0.189 0.588l7.676 5.623-2.958 9.135c-0.070 0.216 0.007 0.454 0.19 '
 '0.589 0.183 0.133 0.431 0.133 0.614 0l7.725-5.641 7.709 5.641c0.092 0.067 '
 '0.199 0.101 0.307 0.101s0.216-0.033 0.307-0.101c0.184-0.135 0.26-0.371 '
 '0.19-0.589l-2.958-9.134 7.692-5.623c0.183-0.133 0.259-0.371 '
 '0.19-0.588z"></path>\n'
 '            <path class="rated" d="M55.437 '
 '10.888c-0.070-0.216-0.271-0.363-0.497-0.363h-9.501l-2.941-9.084c-0.070-0.216-0.271-0.363-0.497-0.364s-0.426 '
 '0.147-0.497 0.362l-

 '= function (t, n) { return new q(n, t) }, V.a.c.fontdeck = function (t, n) { '
 'return new Q(n, t) }, V.a.c.monotype = function (t, n) { return new $(n, t) '
 '}, V.a.c.typekit = function (t, n) { return new J(n, t) }, V.a.c.google = '
 'function (t, n) { return new R(n, t) }; var X = { load: i(V.load, V) }; '
 '"function" == typeof define && define.amd ? define(function () { return X }) '
 ': "undefined" != typeof module && module.exports ? module.exports = X : '
 '(window.WebFont = X, window.WebFontConfig && V.load(window.WebFontConfig)) '
 '}();\n'
 '\n'
 '    WebFont.load({\n'
 '        custom: {\n'
 '            families: ["FontBrandedPrimary:n4,n7", '
 '"FontUniversalPrimary:n4,n7"]\n'
 '        },\n'
 '        timeout: 3000\n'
 '    })\n'
 '</script>\n'
 '<style type="text/css">\n'
 '        \n'
 '            @font-face {\n'
 '                font-family: "FontBrandedPrimary";\n'
 '                src: '
 "url('//s3.images-iherb.com/static/f/lato-regular-v2.eot');\n"
 '      

 '.btn-ghost-primary[disabled]:active,header '
 '.btn-ghost-primary[disabled]:focus,header '
 '.btn-ghost-primary[disabled]:hover{border-color:#f38a00;background-image:-moz- '
 'oldlinear-gradient(top,transparent 0,transparent '
 '100%);background-image:linear-gradient(to bottom,transparent 0,transparent '
 '100%);background-repeat:repeat-x}footer .btn-ghost-primary-darker,header '
 '.btn-ghost-primary-darker{color:#ef6c00;border-color:#ef6c00;background-image:-moz- '
 'oldlinear-gradient(top,transparent 0,transparent '
 '100%);background-image:linear-gradient(to bottom,transparent 0,transparent '
 '100%);background-repeat:repeat-x}footer '
 '.btn-ghost-primary-darker.active,footer '
 '.btn-ghost-primary-darker.focus,footer '
 '.btn-ghost-primary-darker:active,footer '
 '.btn-ghost-primary-darker:focus,footer '
 '.btn-ghost-primary-darker:hover,header '
 '.btn-ghost-primary-darker.active,header '
 '.btn-ghost-primary-darker.focus,header '
 '.btn-ghost-primary-darker:active,header '
 '.

 '.text-secondary-darker{color:#33691e}footer '
 'a.text-secondary-darker:focus,footer a.text-secondary-darker:hover,header '
 'a.text-secondary-darker:focus,header '
 'a.text-secondary-darker:hover{color:#204113}footer .text-success,header '
 '.text-success{color:#458500}footer a.text-success:focus,footer '
 'a.text-success:hover,header a.text-success:focus,header '
 'a.text-success:hover{color:#2b5200}footer .text-info,header '
 '.text-info{color:#60a8ee}footer a.text-info:focus,footer '
 'a.text-info:hover,header a.text-info:focus,header '
 'a.text-info:hover{color:#328fe9}footer .text-warning,header '
 '.text-warning{color:#f38a00}footer a.text-warning:focus,footer '
 'a.text-warning:hover,header a.text-warning:focus,header '
 'a.text-warning:hover{color:#c06d00}footer .text-danger,header '
 '.text-danger{color:#f10200}footer a.text-danger:focus,footer '
 'a.text-danger:hover,header a.text-danger:focus,header '
 'a.text-danger:hover{color:#be0200}footer .text-danger-lighter,header 

...

 "                        'NotDefined',\n"
 '                        null,\n'
 "                        '22',\n"
 "                        'SG$11.69');\n"
 '                \n'
 "                    ih.ga.prod.addImp('ACO-00213',\n"
 "                        'Acure, Brilliantly Brightening, Glowing Serum, 1 fl "
 "oz (30 ml)',\n"
 "                        'Search',\n"
 "                        'Acure',\n"
 "                        'NotDefined',\n"
 '                        null,\n'
 "                        '23',\n"
 "                        'SG$23.39');\n"
 '                \n'
 "                    ih.ga.prod.addImp('ACO-02113',\n"
 "                        'Acure, Deodorant, Lavender &amp; Coconut, 2.25 oz "
 "(63.78 g)',\n"
 "                        'Search',\n"
 "                        'Acure',\n"
 "                        'NotDefined',\n"
 '                        null,\n'
 "                        '24',\n"
 "                        'SG$11.00');\n"
 '                \n'
 "      

 '                                    <input disabled type="checkbox" '
 'id="FilterArgan4653" class="checkbox-filter defer-enable" value="4653" '
 '><label class="filter " for="FilterArgan4653" title="Argan" '
 'data-url="/search?sug=acure+organics&kw=acure+organics&rank=0&noi=96&cids=4653">Argan</label>\n'
 '                                </div>\n'
 '                            </li>\n'
 '                            <li class="filter-item" data-keyword="Body &amp; '
 'Massage Oils">\n'
 '                                <div class="filter-name" data-count="3">\n'
 '                                    <input disabled type="checkbox" '
 'id="FilterBody&MassageOils2063" class="checkbox-filter defer-enable" '
 'value="2063" ><label class="filter " for="FilterBody&MassageOils2063" '
 'title="Body & Massage Oils" '
 'data-url="/search?sug=acure+organics&kw=acure+organics&rank=0&noi=96&cids=2063">Body '
 '& Massage Oils</label>\n'
 '                                </div>\n'
 '              

 'xlink:href="#icon-stars_10"></use></svg></label>\n'
 '                                </div>\n'
 '                            </li>\n'
 '                    </ul>\n'
 '                </fieldset>\n'
 '            </section>\n'
 '            <section id="PriceFilters" class="filter-section '
 'filter-attributes-container">\n'
 '                <fieldset>\n'
 '                    <legend class="filter-header">\n'
 '                        Price\n'
 '                    </legend>\n'
 '                    <div class="price-range">\n'
 '                        <input disabled class="range-min defer-enable" '
 'type="text" placeholder="Min" value="" '
 'data-url="/search?sug=acure+organics&kw=acure+organics&rank=0&noi=96&pmin=" '
 'aria-label="to input" /><span>to</span><input disabled class="range-max '
 'defer-enable" type="text" placeholder="Max" value="" data-url="&pmax=" '
 'aria-label="Max input" />\n'
 '                        <button class="range-filter '
 'icon-recentlyviewedarrow

 '><label class="filter " for="Filter0.0-0.05kg1" title="0.0 - 0.05 kg" '
 'data-url="/search?sug=acure+organics&kw=acure+organics&rank=0&noi=96&weightFilters=1">0.0 '
 '- 0.05 kg</label>\n'
 '                                </div>\n'
 '                            </li>\n'
 '                            <li class="filter-item">\n'
 '                                <div class="filter-name" data-count="13">\n'
 '                                    <input disabled type="checkbox" '
 'id="Filter0.05-0.1kg2" class="checkbox-filter defer-enable" value="2" '
 '><label class="filter " for="Filter0.05-0.1kg2" title="0.05 - 0.1 kg" '
 'data-url="/search?sug=acure+organics&kw=acure+organics&rank=0&noi=96&weightFilters=2">0.05 '
 '- 0.1 kg</label>\n'
 '                                </div>\n'
 '                            </li>\n'
 '                            <li class="filter-item">\n'
 '                                <div class="filter-name" data-count="10">\n'
 '                              

 'data-ga-event="click" data-ga-event-category="Ecommerce" '
 'data-ga-event-label="Click - Get Help Now" data-ga-event-action="Contact '
 'US">\n'
 '                                    Get Help Now &raquo;\n'
 '                                </a>\n'
 '                            </li>\n'
 '                        </ul>\n'
 '                </div>\n'
 '            </div>\n'
 '        </div>\n'
 '        <div class="container-fluid">\n'
 '            <div class="row">\n'
 '                    <div class="separator"></div>\n'
 '                <div class="col-xs-24 footer-bottom-text">\n'
 '                    <p>\n'
 '                        <input type="hidden" '
 'class="model-properties-clock-footer"\n'
 '                               data-year="2019"\n'
 '                               data-month="10"\n'
 '                               data-day="28"\n'
 '                               data-hour="9"\n'
 '                               data-minute="11"\n'
 '                        